# TDA and HVG benchmark measures of chaos from a time series

We consider the point summaries from Myers _et. al._ (2019) "Persistent homology of complex networks for dynamic state detection". These are the maximum persistence ratio $P(D)$, the homology class ratio $M(D)$, and the normalised persistent entropy $E'(D)$, of the persistence diagram $D$ of either the $k$-nearest neighbour graph of a delay embedding of the time series, or the ordinal partition network of the time series.

We also consider the HVG measure from Hasson _et. al._ (2018) "A combinatorial framework to quantify peak/pit asymmetries in complex dynamics". This is the $L_1$ distance between the degree distributions of the _top_ and _bottom_ HVG graphs.

## Set up data and functions

In [1]:
# patch bug in Teaspoon's `teaspoon.parameter_selection.MsPE`
# can cause a reference error by not defining the delay peak

from teaspoon.parameter_selection import MsPE
from PATCH_MsPE_tau import MsPE_tau
MsPE.MsPE_tau = MsPE_tau

In [2]:
# patch bug in Teaspoon's `teaspoon.TDA.PHN.DistanceMatrix`
# can crash when the input graph is not connected

from teaspoon.TDA import PHN
from PATCH_DistanceMatrix import DistanceMatrix
PHN.DistanceMatrix = DistanceMatrix

In [3]:
# patch bug in Teaspoon's `teaspoon.SP.network`
# can crash due to tsa_tools not being defined

from teaspoon.SP import tsa_tools
from teaspoon.SP import network
network.tsa_tools = tsa_tools

In [4]:
import pickle
import numpy as np
from functools import partial
from teaspoon.SP.network import ordinal_partition_graph
from teaspoon.SP.network import knn_graph
from teaspoon.SP.network import ordinal_partition_graph
from gtda.time_series import takens_embedding_optimal_parameters
from teaspoon.SP.network_tools import remove_zeros
from teaspoon.TDA.PHN import PH_network
from teaspoon.TDA.PHN import point_summaries
from trajectories import generate_trajectories
from plots_and_correlates import plot_lce_estimate_and_correlation
from matplotlib import pyplot as plt
from scipy import stats
from lca_supervised_learning import score_classification
from lca_supervised_learning import score_regression
from lca_supervised_learning import score_regression_pos
from lca_supervised_learning import score_regression_KNN
from lca_supervised_learning import score_regression_pos_KNN


In [5]:
import ipyparallel as ipp
clients = ipp.Client()
dv = clients.direct_view()
lbv = clients.load_balanced_view()

In [6]:
SEED = 42
SAMPLES = 500
LENGTH = 500
LENGTHS = [LENGTH]
experimental_data_all = {}
for length in LENGTHS:
    experimental_data_all |= {
        (SEED, length, SAMPLES): generate_trajectories(
            RANDOM_SEED=SEED, TS_LENGTH=length, CONTROL_PARAM_SAMPLES=SAMPLES
        )
    }


Experiment config -- SEED:42, LENGTH:500, SAMPLES:500


In [7]:
time_series_length = LENGTH
experimental_data = experimental_data_all[42, time_series_length, 500]

In [8]:
logistic_trajectories = experimental_data["logistic"]["trajectories"]
logistic_lces = experimental_data["logistic"]["lces"]
logistic_control_params = experimental_data["logistic"]["sys_params"]

In [9]:
henon_trajectories = experimental_data["henon"]["trajectories"]
henon_lces = experimental_data["henon"]["lces"]
henon_control_params = experimental_data["henon"]["sys_params"]

In [10]:
ikeda_trajectories = experimental_data["ikeda"]["trajectories"]
ikeda_lces = experimental_data["ikeda"]["lces"]
ikeda_control_params = experimental_data["ikeda"]["sys_params"]

In [11]:
tinkerbell_trajectories = experimental_data["tinkerbell"]["trajectories"]
tinkerbell_lces = experimental_data["tinkerbell"]["lces"]
tinkerbell_control_params = experimental_data["tinkerbell"]["sys_params"]

## Compute TDA estimates and correlations with Lyapunov exponents

In [12]:
def generate_tda_estimates(
    sys_name,
    param_name,
    trajectories,
    control_params,
    actual_lces,
    show_plot=False,
    compute_knn=True,
    compute_opn=True,
):
    # store results to be returned
    correlations_and_scores = {}

    # embedding required for k-NN graph representation
    takens_optimal = partial(
        takens_embedding_optimal_parameters, max_dimension=8, max_time_delay=50
    )
    optimal_embedding_params = lbv.map_sync(takens_optimal, trajectories)

    if compute_knn:
        # compute the k-NN graphs
        knn_graphs = []
        for ts, embed in zip(trajectories, optimal_embedding_params):
            tau, dim = embed
            graph = knn_graph(ts, n=dim, tau=tau, k=4)  # k=4 in original paper
            graph = remove_zeros(graph)
            knn_graphs.append(graph)

        # from graphs compute the persistence diagrams
        distance_matrices = lbv.map_sync(DistanceMatrix, knn_graphs)  # ipyparallel
        knn_diagrams = lbv.map_sync(PH_network, distance_matrices)  # ipyparallel

        # from diagrams compute the point summary statistics
        knn_stats = np.array(
            [
                point_summaries(diagram, A)
                for diagram, A in zip(knn_diagrams, knn_graphs)
            ]
        )
        knn_stats = np.nan_to_num(knn_stats)
        (
            max_persistence_ratio,
            persistent_entropy_normalised,
            homology_class_ratio,
        ) = knn_stats.T

        # dictionary of our k-NN based estimates
        point_summary_values = {
            r"$k$-NN graph $P(D)$": max_persistence_ratio,
            r"$k$-NN graph $E'(D)$": persistent_entropy_normalised,
            r"$k$-NN graph $M(D)$": homology_class_ratio,
        }

        # compute the correlations of the estimates with the actual lyapunov values
        for estimate_name, estimates in point_summary_values.items():
            sequence_length = len(trajectories[0]) - 1
            correlations_and_scores[
                estimate_name, sys_name, sequence_length
            ] = plot_lce_estimate_and_correlation(
                estimate_name,
                sys_name,
                param_name,
                estimates,
                actual_lces,
                control_params,
                len(trajectories[0]) - 1,
                sharey=False,
                show_plot=show_plot,
                plot_actual=True
            )
            correlations_and_scores[
                estimate_name, sys_name, sequence_length
            ] |= {
                "classification_f1": score_classification(estimates.reshape(-1,1), actual_lces),
                "regression_neg_mean_absolute": score_regression(estimates.reshape(-1,1), actual_lces),
                "pos_regression_neg_mean_absolute": score_regression_pos(estimates.reshape(-1,1), actual_lces),
                "regression_neg_mean_absolute_poly": score_regression_KNN(estimates.reshape(-1,1), actual_lces),
                "pos_regression_neg_mean_absolute_poly": score_regression_pos_KNN(estimates.reshape(-1,1), actual_lces),
            }

    if compute_opn:
        # compute the ordinal partition graphs
        opn_graphs = []
        for ts in trajectories:
            try:
                graph = ordinal_partition_graph(ts)
            except ValueError as err:
                message = getattr(err, "message", repr(err))
                if "negative dimensions are not allowed" in message:
                    print(
                        "WARNING: ordinal_partition_graph() tried to use a negative dimension"
                    )
                    graph = np.array([[0]])
                else:
                    raise err
            graph = remove_zeros(graph)
            opn_graphs.append(graph)

        # from graphs compute the persistence diagrams
        distance_matrices_opn = lbv.map_sync(DistanceMatrix, opn_graphs)
        opn_diagrams = lbv.map_sync(PH_network, distance_matrices_opn)

        # ensure diagrams are not empty
        opn_graphs_2d = []
        for G in opn_graphs:
            if len(G) > 0:
                opn_graphs_2d.append(G)
            else:
                opn_graphs_2d.append(np.array([[0]]))

        # from diagrams compute the point summary statistics
        knn_stats_opn = np.array(
            [
                point_summaries(diagram, A)
                for diagram, A in zip(opn_diagrams, opn_graphs_2d)
            ]
        )
        knn_stats_opn = np.nan_to_num(knn_stats_opn)
        (
            max_persistence_ratio_opn,
            persistent_entropy_normalised_opn,
            homology_class_ratio_opn,
        ) = knn_stats_opn.T

        # dictionary of our ordinal partition based estimates
        point_summary_values_opn = {
            r"Ordinal graph $P(D)$": max_persistence_ratio_opn,
            r"Ordinal graph $E'(D)$": persistent_entropy_normalised_opn,
            r"Ordinal graph $M(D)$": homology_class_ratio_opn,
        }

        # compute the correlations of the estimates with the actual lyapunov values
        for estimate_name, estimates in point_summary_values_opn.items():
            sequence_length = len(trajectories[0]) - 1
            correlations_and_scores[
                estimate_name, sys_name, sequence_length
            ] = plot_lce_estimate_and_correlation(
                estimate_name,
                sys_name,
                param_name,
                estimates,
                actual_lces,
                control_params,
                len(trajectories[0]) - 1,
                sharey=False,
                show_plot=show_plot,
                plot_actual=True
            )
            correlations_and_scores[
                estimate_name, sys_name, sequence_length
            ] |= {
                "classification_f1": score_classification(estimates.reshape(-1,1), actual_lces),
                "regression_neg_mean_absolute": score_regression(estimates.reshape(-1,1), actual_lces),
                "pos_regression_neg_mean_absolute": score_regression_pos(estimates.reshape(-1,1), actual_lces),
                "regression_neg_mean_absolute_poly": score_regression_KNN(estimates.reshape(-1,1), actual_lces),
                "pos_regression_neg_mean_absolute_poly": score_regression_pos_KNN(estimates.reshape(-1,1), actual_lces),
            }


    return correlations_and_scores


In [13]:
all_results_tda = {}
for sys_info in [
    ["Logistic", "r", logistic_trajectories, logistic_control_params, logistic_lces],
    ["Hénon", "a", henon_trajectories, henon_control_params, henon_lces],
    ["Tinkerbell", "a", tinkerbell_trajectories, tinkerbell_control_params, tinkerbell_lces],
    ["Ikeda", "a", ikeda_trajectories, ikeda_control_params, ikeda_lces],
]:
    all_results_tda |= generate_tda_estimates(*sys_info)

In [14]:
with open(f"outputs/data/TDA_benchmarks_{time_series_length}.pkl", "wb") as file:
    pickle.dump(all_results_tda, file)

for result in all_results_tda:
    print(result, all_results_tda[result])

('$k$-NN graph $P(D)$', 'Logistic', 500) {'spearmanr': SpearmanrResult(correlation=0.5269891749080795, pvalue=4.437390937534617e-37), 'pos_spearmanr': SpearmanrResult(correlation=0.3884951514079341, pvalue=3.5620111823922e-15), 'pearsonr': (0.7067373839658496, 7.234544640739665e-77), 'pos_pearsonr': (0.3798898024386352, 1.5831693793718047e-14), 'classification_f1': 0.9801666083168395, 'regression_neg_mean_absolute': -0.10267653128322765, 'pos_regression_neg_mean_absolute': -0.15289213085930337, 'regression_neg_mean_absolute_poly': -0.1525217659331251, 'pos_regression_neg_mean_absolute_poly': -0.15536919180820083}
("$k$-NN graph $E'(D)$", 'Logistic', 500) {'spearmanr': SpearmanrResult(correlation=0.08319621943283917, pvalue=0.06304251345209909), 'pos_spearmanr': SpearmanrResult(correlation=-0.6490167025073115, pvalue=6.229536589179508e-47), 'pearsonr': (0.7003135668924785, 6.291389944617876e-75), 'pos_pearsonr': (-0.34387065320727483, 5.141161252466076e-12), 'classification_f1': 0.97992

## Compute HVG estimates and correlations with Lyapunov exponents

In [15]:
from ts2vg import HorizontalVG as HVG
from scipy.stats import wasserstein_distance


def generate_hvg_estimates(
    sys_name,
    param_name,
    trajectories,
    control_params,
    actual_lces,
    show_plot=False,
):
    # store results to be returned
    correlations_and_scores = {}

    hvg_top = [
        HVG(directed="left_to_right").build(ts, only_degrees=True)
        for ts in trajectories
    ]
    hvg_bot = [
        HVG(directed="left_to_right").build(-1 * ts, only_degrees=True)
        for ts in trajectories
    ]

    hvg_top_deg_dist = [hvg.degree_distribution for hvg in hvg_top]
    hvg_bot_deg_dist = [hvg.degree_distribution for hvg in hvg_bot]

    def deg_dist_l1(dist1, dist2):
        """Compute $L_1$ distance between distributions.

        Assumes each distribution `dist` is a pair `(ks, ps)` where `ps[i]` is the
        sample probability of value `ks[i]`.
        """
        ks1, ps1 = dist1
        ks2, ps2 = dist2

        prob1 = {k: p for k, p in zip(ks1, ps1)}
        prob2 = {k: p for k, p in zip(ks2, ps2)}

        ks = list(set(list(ks1) + list(ks2)))

        delta = 0
        for k in ks:
            delta += abs(prob1.get(k, 0) - prob2.get(k, 0))

        return delta

    def deg_dist_w1(dist1, dist2):
        """Compute Wasserstein distance between distributions.

        Assumes each distribution `dist` is a pair `(ks, ps)` where `ps[i]` is the
        sample probability of value `ks[i]`.
        """
        ks1, ps1 = dist1
        ks2, ps2 = dist2

        return wasserstein_distance(ks1, ks2, ps1, ps2)

    delta_vga_l1 = np.array([
        deg_dist_l1(dist1, dist2)
        for dist1, dist2 in zip(hvg_top_deg_dist, hvg_bot_deg_dist)
    ])
    delta_vga_w1 = np.array([
        deg_dist_w1(dist1, dist2)
        for dist1, dist2 in zip(hvg_top_deg_dist, hvg_bot_deg_dist)
    ])


    # compute the correlations of the estimates with the actual lyapunov values
    for estimate_name, estimates in {
        r"$\Delta$VGA ($L_1$)": delta_vga_l1,
        r"$\Delta$VGA ($W_1$)": delta_vga_w1,
    }.items():
        sequence_length = len(trajectories[0]) - 1
        correlations_and_scores[
            estimate_name, sys_name, sequence_length
        ] = plot_lce_estimate_and_correlation(
            estimate_name,
            sys_name,
            param_name,
            estimates,
            actual_lces,
            control_params,
            sequence_length,
            sharey=False,
            show_plot=show_plot,
            plot_actual=True,
        )
        correlations_and_scores[
            estimate_name, sys_name, sequence_length
        ] |= {
            "classification_f1": score_classification(estimates.reshape(-1,1), actual_lces),
            "regression_neg_mean_absolute": score_regression(estimates.reshape(-1,1), actual_lces),
            "pos_regression_neg_mean_absolute": score_regression_pos(estimates.reshape(-1,1), actual_lces),
            "regression_neg_mean_absolute_poly": score_regression_KNN(estimates.reshape(-1,1), actual_lces),
            "pos_regression_neg_mean_absolute_poly": score_regression_pos_KNN(estimates.reshape(-1,1), actual_lces),
        }

    return correlations_and_scores


In [16]:
all_results_hvg = {}
for sys_info in [
    ["Logistic", "r", logistic_trajectories, logistic_control_params, logistic_lces],
    ["Hénon", "a", henon_trajectories, henon_control_params, henon_lces],
    ["Tinkerbell", "a", tinkerbell_trajectories, tinkerbell_control_params, tinkerbell_lces],
    ["Ikeda", "a", ikeda_trajectories, ikeda_control_params, ikeda_lces],
]:
    all_results_hvg |= generate_hvg_estimates(*sys_info)


In [17]:
with open(f"outputs/data/HVG_benchmarks_{time_series_length}.pkl", "wb") as file:
    pickle.dump(all_results_hvg, file)


for result in all_results_hvg:
    print(result, all_results_hvg[result])

('$\\Delta$VGA ($L_1$)', 'Logistic', 500) {'spearmanr': SpearmanrResult(correlation=0.7169684266159593, pvalue=4.565533121861757e-80), 'pos_spearmanr': SpearmanrResult(correlation=0.6933453122710281, pvalue=6.879237369905276e-56), 'pearsonr': (0.3828372375853202, 6.709629490221069e-19), 'pos_pearsonr': (0.7022820209029834, 6.779376515587827e-58), 'classification_f1': 0.7622551102905918, 'regression_neg_mean_absolute': -0.16158338610659764, 'pos_regression_neg_mean_absolute': -0.09796175973167569, 'regression_neg_mean_absolute_poly': -0.1692025967758911, 'pos_regression_neg_mean_absolute_poly': -0.09082448514767155}
('$\\Delta$VGA ($W_1$)', 'Logistic', 500) {'spearmanr': SpearmanrResult(correlation=0.802742520625107, pvalue=6.900512079155294e-114), 'pos_spearmanr': SpearmanrResult(correlation=0.8520427884540193, pvalue=1.3871041127627756e-108), 'pearsonr': (0.63529187728276, 7.219348336475119e-58), 'pos_pearsonr': (0.8514718611401593, 2.71632626175277e-108), 'classification_f1': 0.62508